Day 6 (https://adventofcode.com/2023/day/6)

In [1]:
with open('./inputs/day06.txt', 'r') as f:
    lines = f.readlines()
    races = [
        {
            'time': int(l0),
            'dist': int(l1)
        } for l0, l1 in zip(lines[0].split()[1:], lines[1].split()[1:])
    ]

    # function to calculate dist travelled
    def get_dist(total_time, button_time):
        return (total_time - button_time) * button_time
    
    # find num ways to win
    part1_ans = 1
    for race in races:
        part1_ans *= sum((get_dist(race['time'], i) > race['dist'] for i in range(race['time']+1)))

    print('Answer to Day 6, Part 1:', part1_ans)

    race = {
        'time': int(''.join(lines[0].split()[1:])),
        'dist': int(''.join(lines[1].split()[1:]))
    }

    # takes a few seconds, but I can just use the same method
    print('Answer to Day 6, Part 2:', sum((get_dist(race['time'], i) > race['dist'] for i in range(race['time']+1))))

Answer to Day 6, Part 1: 861300
Answer to Day 6, Part 2: 28101347


Day 7 (https://adventofcode.com/2023/day/7)

In [2]:
import functools
import itertools

@functools.total_ordering
class Hand:
    card_ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
    def __init__(self, hand_str):
        self.cards = [Hand.card_ranks.index(c) for c in hand_str]

    @property
    def hand_type(self):
        card_set = set(self.cards)
        # 5 of a kind
        if len(card_set) == 1:
            return 6
        # 4 of a kind
        if any((self.cards.count(c)==4 for c in card_set)):
            return 5
        # full house
        if any((self.cards.count(c)==3 for c in card_set)) and any((self.cards.count(c)==2 for c in card_set)):
            return 4
        # 3 of a kind
        if any((self.cards.count(c)==3 for c in card_set)):
            return 3
        # 2 pair
        if sum((self.cards.count(c)==2 for c in card_set)) == 2:
            return 2
        # 1 pair
        if any((self.cards.count(c)==2 for c in card_set)):
            return 1
        # high card
        else:
            return 0
        
    def __repr__(self):
        return str([Hand.card_ranks[i] for i in self.cards])
        
    def __eq__(self, other):
        return all(c1==c2 for c1, c2 in zip(self.cards, other.cards))
    
    def __gt__(self, other):
        if self.hand_type > other.hand_type:
            return True
        elif self.hand_type == other.hand_type:
            return self.cards > other.cards
        else:
            return False
        
@functools.total_ordering
class Hand2:
    card_ranks = ['J', '2', '3', '4', '5', '6', '7', '8', '9', 'T', 'Q', 'K', 'A']
    def __init__(self, hand_str):
        self.cards = [Hand2.card_ranks.index(c) for c in hand_str]

    def orig_hand_type(cards):
        return Hand(''.join((Hand2.card_ranks[i] for i in cards))).hand_type
    
    @property
    def hand_type(self):
        wilds_iter = itertools.combinations_with_replacement(range(1, len(Hand2.card_ranks)), self.cards.count(0))
        return max(list(Hand2.orig_hand_type(list(filter(lambda x: x!=0, self.cards)) + list(wilds)) for wilds in wilds_iter))

    def __repr__(self):
        return str([Hand.card_ranks[i] for i in self.cards])
        
    def __eq__(self, other):
        return all(c1==c2 for c1, c2 in zip(self.cards, other.cards))
    
    def __gt__(self, other):
        if self.hand_type > other.hand_type:
            return True
        elif self.hand_type == other.hand_type:
            return self.cards > other.cards
        else:
            return False

with open('./inputs/day07.txt', 'r') as f:
    lines = [{'hand': l.split()[0], 'bid': int(l.split()[1])} for l in f.readlines()]
    print('Answer to Day 7, Part 1:', sum(((i+1)*l['bid']) for i, l in enumerate(sorted(lines, key=lambda x: Hand(x['hand'])))))
    print('Answer to Day 7, Part 2:', sum(((i+1)*l['bid']) for i, l in enumerate(sorted(lines, key=lambda x: Hand2(x['hand'])))))

Answer to Day 7, Part 1: 250957639
Answer to Day 7, Part 2: 251515496


Day 8 (https://adventofcode.com/2023/day/8)

In [3]:
with open('./inputs/day08.txt', 'r') as f:
    lines = f.readlines()
    directions = lines[0].strip()
    nodes = {n[0:3]: (n[7:10], n[12:15]) for n in lines[2:]}

    num_steps = 0
    current_location = 'AAA'
    while current_location != 'ZZZ':
        for step in directions:
            current_location = nodes[current_location][0 if step == 'L' else 1]
            num_steps += 1

    print('Answer to Day 8, Part 1:', num_steps)

    # not sure that this method works generally. but hey, it got me the answer 🤷‍♂️
    part2_ans = len(directions)
    for pos in filter(lambda x: x[-1] == 'A', nodes.keys()):
        num_iterations = 0
        while pos[-1] != 'Z':
            for step in directions:
                pos = nodes[pos][0 if step == 'L' else 1]
            
            num_iterations += 1

        part2_ans*=num_iterations

    print('Answer to Day 8, Part 2:', part2_ans)

Answer to Day 8, Part 1: 13301
Answer to Day 8, Part 2: 7309459565207
